In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


In [2]:
#np.random.seed(404)
np.set_printoptions(precision=2, suppress=True)

In [3]:
# Read data
data = pd.read_excel('Modelling_Data_Phase_2.xlsx', sheet_name ='Seed', index_col=False) #change the sheet name according to the tabs to run the various cycles in the current Phase
data.head()


,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SHA,XCO
0,0.1010,0.478,0.213,0.208,260,2.0,24000,158.555963,0.144916,0.130594
1,0.1010,0.202,0.480,0.217,260,2.0,24000,220.116161,0.107552,0.237733
2,0.1170,0.421,0.104,0.360,260,2.0,24000,242.502579,0.157901,0.165786
3,0.0966,0.201,0.230,0.465,260,2.0,24000,270.214203,0.152576,0.195074
4,0.1280,0.112,0.401,0.359,260,2.0,24000,278.589571,0.164758,0.189697


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zr      50 non-null     float64
 1   Cu      50 non-null     float64
 2   Co      50 non-null     float64
 3   Fe      50 non-null     float64
 4   T       50 non-null     int64  
 5   H2:CO   50 non-null     float64
 6   GHSV    50 non-null     int64  
 7   STY     50 non-null     float64
 8   SHA     50 non-null     float64
 9   XCO     50 non-null     float64
dtypes: float64(8), int64(2)
memory usage: 4.0 KB


In [5]:
# General stastistical data
data.describe()

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY,SHA,XCO
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.166729,0.189797,0.197736,0.445650,260.000000,2.120000,25080.000000,227.207572,0.121363,0.275048
std,0.162769,0.166326,0.117408,0.224635,9.035079,0.458035,4724.404724,123.742048,0.024084,0.198131
min,0.048322,0.049333,0.049509,0.048463,240.000000,1.500000,12000.000000,13.552616,0.057141,0.000000
25%,0.094251,0.099046,0.116681,0.234844,260.000000,2.000000,24000.000000,127.577012,0.109250,0.131122
50%,0.101000,0.109500,0.200430,0.392471,260.000000,2.000000,24000.000000,237.118775,0.124374,0.218833
75%,0.128000,0.249706,0.214183,0.646539,260.000000,2.000000,24000.000000,341.004415,0.141623,0.398233
max,0.846036,0.663680,0.480000,0.806660,280.000000,3.000000,42000.000000,476.519720,0.164758,0.940109


In [6]:
# Define X and Y from the data

X = data[['Zr ', 'Cu ', 'Co ', 'Fe','T','H2:CO','GHSV']]
Y = data['STY']
Y2 = pd.DataFrame({'YHA': data['SHA'] * data['XCO']})
Y3 = pd.concat([Y, Y2], axis=1)

In [7]:
#Check data for X or Y
Y

0     158.555963
1     220.116161
2     242.502579
3     270.214203
4     278.589571
5     304.800028
6      44.529302
7      74.679922
8     123.075373
9     176.464488
10    315.501875
11    341.968038
12     13.552616
13     18.942366
14    104.608601
15    340.585262
16    343.287457
17    387.622672
18     56.125403
19     60.263459
20    261.245758
21    299.888285
22    340.731379
23    377.526782
24    136.987869
25    137.247937
26    178.201845
27    349.171300
28    362.271349
29    376.432302
30     67.569134
31    149.907943
32     51.050119
33    125.915853
34    193.834026
35     94.303809
36    160.876357
37     92.817274
38    254.406612
39    275.743318
40    132.560486
41    476.519720
42    385.667532
43    321.388352
44    341.095427
45    156.095377
46    391.958214
47    231.734972
48    372.719912
49    388.524025
Name: STY, dtype: float64

In [8]:
# Feature normalization for X. 

from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler(feature_range=(0,1))
mm.fit(np.array(X)[:,4:])

def scalex(x):
    x = np.array(x)
    x[:,4:] = mm.transform(x[:,4:])
    return x

def scalex_const(x_const):
    x_const = np.array(x_const)
    x_const = mm.transform(x_const)
    return x_const

X = scalex(X)

In [13]:
# Feature standardization for Y, applied to Y. Invert Y by multiplying with -1 to form a MAXIMIZATION problem as BO is MINIMIZATION by default

from sklearn.preprocessing import StandardScaler

Y=np.array(Y).reshape(-1, 1) 

stc = StandardScaler()
stc.fit(np.array(Y))

def scaley(y):
    y_scaled = -1*stc.transform(np.array(y))
    return y_scaled

def scaley_inv(y_scaled):
    y = stc.inverse_transform(np.array(-1*y_scaled))
    return y

Y = scaley(Y)

In [14]:
#Check the values of X or Y
Y

array([[ 0.56],
       [ 0.06],
       [-0.12],
       [-0.35],
       [-0.42],
       [-0.63],
       [ 1.49],
       [ 1.25],
       [ 0.85],
       [ 0.41],
       [-0.72],
       [-0.94],
       [ 1.74],
       [ 1.7 ],
       [ 1.  ],
       [-0.93],
       [-0.95],
       [-1.31],
       [ 1.4 ],
       [ 1.36],
       [-0.28],
       [-0.59],
       [-0.93],
       [-1.23],
       [ 0.74],
       [ 0.73],
       [ 0.4 ],
       [-1.  ],
       [-1.1 ],
       [-1.22],
       [ 1.3 ],
       [ 0.63],
       [ 1.44],
       [ 0.83],
       [ 0.27],
       [ 1.08],
       [ 0.54],
       [ 1.1 ],
       [-0.22],
       [-0.4 ],
       [ 0.77],
       [-2.04],
       [-1.29],
       [-0.77],
       [-0.93],
       [ 0.58],
       [-1.34],
       [-0.04],
       [-1.19],
       [-1.32]])

In [15]:
# Import necesarry libraries for Gaussian process regression 

from gpflow.models import GPR
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.optimizers import Scipy
from gpflow.kernels import SquaredExponential as SE, Constant as C, White as W, SharedIndependent as SI
from gpflow.inducing_variables import SharedIndependentInducingVariables as SIIV, InducingPoints as IP
from sklearn.metrics import r2_score, mean_squared_error

In [16]:
#Define the kernels using squared exponential. The dimentions of lengthscales must match the number of input features
#This is a single objective task with 7 input features, where each feature correspond to the metal composition and reaction conditions

# single objective with 4 metals + reaction conditions
kernel = SE(lengthscales=[0.1,0.1,0.1,0.1,0.1,0.1,0.1])


# Gaussian process regression
gp_model = GPR((X, Y), kernel=kernel)

# Optimize the lengthscales
opt = Scipy()
opt.minimize(gp_model.training_loss, gp_model.trainable_variables)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 28.039659321260928
        x: [ 6.361e-01  3.487e+01  7.426e-01 -6.892e-01  6.591e-02
             6.182e+01  1.146e+00  1.699e+00 -2.409e+00]
      nit: 78
      jac: [ 1.808e-05 -5.174e-05 -2.658e-05 -3.739e-04  1.368e-05
             3.174e-05  7.518e-06  3.785e-05  1.094e-04]
     nfev: 102
     njev: 102
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>

In [17]:
from sklearn.metrics import r2_score, mean_squared_error

# After optimization, you can make predictions with the trained model
Y_pred_mean, _ = gp_model.predict_y(X)  # Predicted mean values

# Evaluate R^2 score
r2 = r2_score(Y, Y_pred_mean)

# Evaluate mean squared error (RMSE)
mse = mean_squared_error(Y, Y_pred_mean)
rmse = np.sqrt(mse)

# Print the evaluation metrics with limited decimal places

print("R^2 score: {:.2f}".format(r2))
print("Root Mean Squared Error: {:.2f}".format(rmse))

R^2 score: 0.94
Root Mean Squared Error: 0.25


In [18]:
# Check the optimized hyperparameters

optimized_lengthscales = gp_model.kernel.lengthscales.numpy()
print("Optimized Lengthscales:", optimized_lengthscales)

Optimized Lengthscales: [ 1.06 34.87  1.13  0.41  0.73 61.82  1.42]


# Bayesian optimization

In [19]:
from trieste.space import LinearConstraint
from trieste.space import Box

# Define lower and upper bounds for metal fractions and reaction conditions
Zr_lb = 0.1
Zr_ub = 0.1
Cu_lb = 0.05
Cu_ub = 0.2
Co_lb = 0.1
Co_ub = 0.3
Fe_lb = 0.4
Fe_ub = 0.8
T_lb = 250
T_ub = 300
H2_CO_lb = 1
H2_CO_ub = 4
GHSV_lb = 30000
GHSV_ub = 50000

const_lb = -10
const_ub = 10

const_mat = np.array([T_lb, H2_CO_lb, GHSV_lb, T_ub, H2_CO_ub, GHSV_ub]).reshape(2,3)
print(const_mat)
const_mat = scalex_const(const_mat)
print(const_mat)

T_lb = const_mat[0,0]
T_ub = const_mat[1,0]
H2_CO_lb = const_mat[0,1]
H2_CO_ub = const_mat[1,1]
GHSV_lb = const_mat[0,2]
GHSV_ub = const_mat[1,2]


# Define linear constraints. Apply lb and ub to the scalar product of the number vector and the feature vector

# Metal compositions + reaction conditions
constraints = [LinearConstraint(A=tf.constant
       ([[1, 1, 1, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], 
        [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]]), 
                                
        lb=tf.constant([1, Zr_lb, Cu_lb, Co_lb, Fe_lb, T_lb, H2_CO_lb, GHSV_lb]), 
        ub=tf.constant([1, Zr_ub, Cu_ub, Co_ub, Fe_ub, T_ub, H2_CO_ub, GHSV_ub]))]

constrained_search_space = Box([0, 0, 0, 0, const_lb, const_lb, const_lb], 
                               [1, 1, 1, 1, const_ub, const_ub, const_ub], 
                               constraints=constraints)

[[  250     1 30000]
 [  300     4 50000]]
[[ 0.25 -0.33  0.6 ]
 [ 1.5   1.67  1.27]]


In [20]:
# Essential functions for formatting data

from trieste.data import Dataset

def observer(in_):
    in_ = tf.convert_to_tensor(in_)
    out_, _ = gp_model.predict_y(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

def initial_data(in_, out_):
    in_ = tf.convert_to_tensor(in_)
    out_ = tf.convert_to_tensor(out_)
    return Dataset(in_, out_)

In [21]:
# Import necessary libraries to build model

from trieste.models.gpflow import GaussianProcessRegression
from trieste.bayesian_optimizer import BayesianOptimizer
from trieste.acquisition.rule import EfficientGlobalOptimization
from trieste.acquisition.function import Fantasizer
from trieste.acquisition import LocalPenalization
from trieste.acquisition.function import ExpectedHypervolumeImprovement
from trieste.acquisition.function import ExpectedImprovement
from trieste.acquisition.function import PredictiveVariance

#Fit the model
model = GaussianProcessRegression(gp_model, num_kernel_samples=10)

# Define acquisition functions. We use the ei rule for exploitation and pv rule for exploration

ei = ExpectedImprovement(constrained_search_space)
rule_ei = EfficientGlobalOptimization(builder=ei)

pv = PredictiveVariance()
rule_pv = EfficientGlobalOptimization(builder=pv)

# Bayesian optimizer
bo = BayesianOptimizer(observer, constrained_search_space)


In [22]:
# Run the Bayesian optimizer for single objective

batch_size = 5 # This number is user defined and determines the number of recommendation made by the BO. Typically 5-10 generations yield good results. For the project, we used a batch of 30. 

# Alternate between rule_ei or rule_pv as parmaters when runnig the BO for exploitation or exploration campaigns, respectively.

bo_result = bo.optimize(batch_size, initial_data(X, Y), model, rule_pv, track_state = False, fit_initial_model=False)

Optimization completed without errors


In [23]:
# Get results from the Bayesian optimizer

bo_initial_data = bo_result.try_get_final_dataset()
bo_X = bo_result.try_get_final_dataset().query_points.numpy()[-batch_size:,:]

bo_X[:,4:] = mm.inverse_transform(bo_X[:,4:]) 

bo_Y = bo_result.try_get_final_dataset().observations.numpy()[-batch_size:,:]
np.set_printoptions(precision=3, suppress=True)

result=(np.concatenate((bo_X, scaley_inv(bo_Y)), axis=1))


In [25]:
# Create dataframe with results 

dfresult = pd.DataFrame(result, columns = ['Zr','Cu','Co','Fe','T','H2:CO','GHSV','STY'])
dfresult = dfresult.round(2)
dfresult

,Zr,Cu,Co,Fe,T,H2:CO,GHSV,STY
0,0.1,0.05,0.1,0.75,300.00,3.99,49999.95,335.55
1,0.1,0.20,0.3,0.40,300.00,1.00,49999.98,279.03
2,0.1,0.05,0.1,0.75,271.92,4.00,50000.00,487.03
3,0.1,0.05,0.1,0.75,300.00,1.00,30001.52,304.30
4,0.1,0.20,0.3,0.40,300.00,4.00,30000.13,249.61


In [ ]:
#The above dataframe is the output of the current campiagn recommending catalyst composition worth investigation and its predicted yield.
#In the study we performed these experimental recommendation and measured the actual yield